In [ ]:
pip install openai

In [ ]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

## First, let's do it all in 1 cell

In [ ]:
def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response

print(run_conversation()["choices"][0]["message"]["content"])

## Now let's do it step-by-step

In [ ]:
def prompt_gpt_with_functions(messages):
    # Step 1: send the conversation and available functions to GPT
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    print(f"1. Call ChatCompletion API with: '{messages[0]['content']}'" )
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    return response_message
    
def call_function(response_message):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    function_name = response_message["function_call"]["name"]
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])

    print(f"2. OpenAI suggests to invoke: {function_name}.  Calling now..." )
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    return function_name, function_response

def send_function_response_to_gpt(messages):     
    # Step 4: send the info on the function call and function response to GPT
    print(f"3. Call again ChatCompletion with the response of our function:")
    print(f"   {messages[2]['content']}")
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
    )  # get a new response from GPT where it can see the function response
    return second_response


In [ ]:
prompt = "What's the weather like in Boston?"
messages = [{"role": "user", "content": prompt}]

# Step 1: prompt GPT with the user question
gpt_first_response = prompt_gpt_with_functions(messages)
messages.append(gpt_first_response)  # don't forget to extend conversation with assistant's reply

if gpt_first_response.get("function_call"):
    # Step 2: call our function
    function_name, function_response = call_function(gpt_first_response)

    # Step 3: call GPT with the response of our function call
    messages.append({"role": "function", "name": function_name, "content": function_response})  # extend conversation with function response
    gpt_second_response = send_function_response_to_gpt(messages)

    # Step 4: print the final response
    print("4. OpenAI returns the following answer:")
    print(f'   {gpt_second_response["choices"][0]["message"]["content"]}')
else:
    print("[ERROR] ChatGPT didn't select our function and responded instead:")
    print(gpt_first_response)